In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the pandas documentation to find functions or methods you might not have used yet, or ask questions on Stack Overflow and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:

A quarter is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
A recession is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
A recession bottom is the quarter within a recession which had the lowest GDP.
A university town is a city which has a high percentage of university students compared to the total population of the city.
Hypothesis: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (price_ratio=quarter_before_recession/recession_bottom)

The following data files are available for this assignment:

From the Zillow research data site there is housing data for the United States. In particular the datafile for all homes at a city level, City_Zhvi_AllHomes.csv, has median home sale prices at a fine grained level.
From the Wikipedia page on college towns is a list of university towns in the United States which has been copy and pasted into the file university_towns.txt.
From Bureau of Economic Analysis, US Department of Commerce, the GDP over time of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file gdplev.xls. For this assignment, only look at GDP data from the first quarter of 2000 onward.
Each function in this assignment below is worth 10%, with the exception of run_ttest(), which is worth 50%.


In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [5]:
zillow_allhomes_df  = pd.read_csv(r'C:\Users\sjtal\Documents\python_datascience\City_Zhvi_AllHomes.csv')
zillow_allhomes_df.head()

zillow_allhomes_df.loc[:, 'State'] = zillow_allhomes_df.loc[:, 'State'].map(states)


In [12]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    
    state_town_tuple_list = []
    with open(r'C:\Users\sjtal\Documents\python_datascience\university_towns.txt') as fref:
        for line in fref:
            lineWithoutSquare = line.split('[')[0].strip()
            if ':' in lineWithoutSquare and not ('(')  in lineWithoutSquare :
                continue
            else:
                if not ('(')  in lineWithoutSquare:
                    state = lineWithoutSquare
                else:
                    univTown = lineWithoutSquare.split(' (')[0].strip().split(',')[0].strip()
                    state_town_tuple_list.append((state, univTown) )
                
        state_univtown_df = pd.DataFrame(state_town_tuple_list, columns =['State', 'RegionName'])
    
    return state_univtown_df

get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
...,...,...
508,Wisconsin,River Falls
509,Wisconsin,Stevens Point
510,Wisconsin,Waukesha
511,Wisconsin,Whitewater


In [27]:
GDP_qtr = pd.read_excel(r'C:\Users\sjtal\Documents\python_datascience\gdplev.xls', skiprows= 5)
GDP_qtr= GDP_qtr.dropna(axis=0,how='all').dropna(axis=1,how='all').iloc[:,3:]
GDP_qtr.rename(columns={GDP_qtr.columns[0]: "YearQuarters",
                        GDP_qtr.columns[1]: "GDP(in Billions)",
                        GDP_qtr.columns[2]: "GDP(in Billions of chained 2009)"}, 
                        inplace=True
              )
indexFor2000Start = GDP_qtr[GDP_qtr['YearQuarters'] == '2000q1'].index.item()
indexFor2000Start

GDP_qtr = GDP_qtr.loc[indexFor2000Start:]
#GDP_qtr

,YearQuarters,GDP(in Billions),GDP(in Billions of chained 2009),Unnamed: 10
214,2000q1,10031.0,12359.1,No
215,2000q2,10278.3,12592.5,No
216,2000q3,10357.4,12607.7,No
217,2000q4,10472.3,12679.3,No
218,2001q1,10508.1,12643.3,Yes
...,...,...,...,...
275,2015q2,17998.3,16374.2,No
276,2015q3,18141.9,16454.9,No
277,2015q4,18222.8,16490.7,No
278,2016q1,18281.6,16525.0,No


In [33]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    

#  A recession is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
#(use the chained value in 2009 dollars)  
#     For this assignment, only look at GDP data from the first quarter of 2000 onward.
    for i, row in GDP_qtr.iterrows():

        if (GDP_qtr.loc[i+2, 'GDP(in Billions of chained 2009)'] < GDP_qtr.loc[i+1, 'GDP(in Billions of chained 2009)']) & (GDP_qtr.loc[i+1, 'GDP(in Billions of chained 2009)'] < GDP_qtr.loc[i, 'GDP(in Billions of chained 2009)']):
            return GDP_qtr.loc[i+1, 'YearQuarters']
        
    return "No recession"

get_recession_start()


'2008q3'

In [36]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    
    recessionStart = get_recession_start()
    startIter = GDP_qtr[GDP_qtr['YearQuarters']==recessionStart].index.item()

    #Find the index of the start of the recession and start iterating from that onwards to check rise of GDP
    for i, row in GDP_qtr.iterrows():
        if i < startIter:
            continue
        else:
            if (GDP_qtr.loc[i+2, 'GDP(in Billions of chained 2009)'] > GDP_qtr.loc[i+1, 'GDP(in Billions of chained 2009)']) & (GDP_qtr.loc[i+1, 'GDP(in Billions of chained 2009)'] > GDP_qtr.loc[i, 'GDP(in Billions of chained 2009)']):
                return GDP_qtr.loc[i+1, 'YearQuarters']

    return "No recession end"
    
# get_recession_end()

#('2008q3', '2009q3')

'2009q3'

In [62]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    #     A recession bottom is the quarter within a recession which had the lowest GDP.
    recessionStart = get_recession_start()
    recessionEnd = get_recession_end()
    
    startIter = GDP_qtr[GDP_qtr['YearQuarters']==recessionStart].index.item()
    endIter = GDP_qtr[GDP_qtr['YearQuarters']==recessionEnd].index.item()
    
   
    GDP_recession = GDP_qtr.loc[startIter:endIter, 'GDP(in Billions of chained 2009)'].min() #14355.6
    GDP_recession_idx = GDP_qtr.loc[startIter:endIter, 'GDP(in Billions of chained 2009)'].idxmin()
    
    return GDP_qtr.loc[GDP_recession_idx]['YearQuarters'].upper()

get_recession_bottom()   



'2009Q2'

In [38]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    
    zillow_allhomes_df  = pd.read_csv(r'C:\Users\sjtal\Documents\python_datascience\City_Zhvi_AllHomes.csv')
    zillow_allhomes_df.loc[:, 'State'] = zillow_allhomes_df.loc[:, 'State'].map(states)
    temp_zillow = zillow_allhomes_df.copy()

    temp_zillow.set_index(["State","RegionName"], inplace=True)

    #  convert column  names to datetime
    temp_zillow = temp_zillow[temp_zillow.columns[6:]].rename(columns=pd.to_datetime)

    columns_we_need  = temp_zillow.columns
    columns_we_need = columns_we_need[(columns_we_need > '1999-12-31') & (columns_we_need < '2016-09-01')]

    temp_zillow = temp_zillow[columns_we_need].resample('QS',axis=1).mean()


    #Convert dates to Quarter periods
    month_names = temp_zillow.columns
    month_names = pd.PeriodIndex(month_names, freq='Q')

    #Assign Quater names as column names
    temp_zillow.columns = month_names
    
    #Makes ure the data in the periodindex columns are converted to str so that Merge and Join do not throw an error  
    temp_zillow.columns.values.astype('str')
   
    zillow_allhomes_df = temp_zillow
  
    zillow_allhomes_df = zillow_allhomes_df.reset_index()
    zillow_allhomes_df.set_index(['State', 'RegionName'], inplace=True, drop = True)
    
    return zillow_allhomes_df


#convert_housing_data_to_quarters()


,,2000Q1,2000Q2,2000Q3,2000Q4,2001Q1,2001Q2,2001Q3,2001Q4,2002Q1,2002Q2,...,2014Q2,2014Q3,2014Q4,2015Q1,2015Q2,2015Q3,2015Q4,2016Q1,2016Q2,2016Q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,243464.000000,252251.666667,260138.666667,268484.000000,277120.333333,286576.666667,293365.333333,300368.000000,307615.333333,307932.666667,...,492571.000000,499305.333333,506530.666667,513433.666667,522421.333333,533102.000000,542734.333333,551434.666667,560075.666667,567197.0
California,Los Angeles,236294.666667,242379.333333,248779.333333,255336.333333,261622.000000,268508.666667,274942.000000,282457.000000,291412.666667,302555.333333,...,530952.333333,534494.000000,537976.000000,550407.000000,559531.333333,567509.666667,579534.000000,591045.000000,604423.000000,613363.0
Texas,Houston,106242.666667,106365.666667,107054.000000,108204.666667,108867.333333,108614.000000,108833.000000,109410.333333,110188.000000,111795.333333,...,140745.666667,145022.000000,149337.333333,152946.333333,156465.000000,158326.666667,160039.000000,162204.000000,163464.333333,165267.0
Illinois,Chicago,157851.000000,162824.333333,167271.666667,171826.333333,176428.333333,181054.666667,185542.000000,190079.666667,194006.666667,197951.000000,...,199324.666667,202455.333333,205087.333333,206689.666667,209998.666667,212235.333333,213732.000000,217486.000000,220779.666667,223351.5
Texas,San Antonio,102291.000000,101794.333333,100507.666667,100411.666667,100022.000000,99548.333333,99589.666667,99274.666667,99337.333333,100250.000000,...,134430.666667,136107.333333,138042.333333,140489.666667,142810.000000,145803.666667,148585.333333,150489.000000,153440.333333,155199.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Minnesota,Winton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,80003.000000,81083.666667,80732.000000,83166.333333,85120.666667,87088.333333,87195.333333,83964.333333,82354.666667,79129.0
Mississippi,Eastabuchie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,63276.000000,63448.333333,64417.666667,66019.333333,66523.333333,66922.000000,66960.000000,66296.000000,66313.333333,65820.0
Texas,Dean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,132332.666667,133835.333333,136007.000000,137525.000000,138844.333333,141260.666667,141728.333333,143110.333333,145840.666667,148203.5


In [41]:
#For calendar with 2018 quarter starting in April 2017
per = pd.Period('2018Q1', freq='Q-MAR')
per.start_time
#Timestamp('2017-04-01 00:00:00')
# per.qyear
# #2018
# per.year
#2017

Timestamp('2017-04-01 00:00:00')

In [57]:
# univ_towns = get_list_of_university_towns().assign(MarkUniv='UnivTown').set_index(["State", "RegionName"]).sort_index(inplace=True)
univ_towns = (get_list_of_university_towns().assign(MarkUniv='UnivTown')
              .set_index(["State", "RegionName"])
             )

univ_towns

MarkUniv
State     RegionName             
Alabama   Auburn         UnivTown
          Florence       UnivTown
          Jacksonville   UnivTown
          Livingston     UnivTown
          Montevallo     UnivTown
...                           ...
Wisconsin River Falls    UnivTown
          Stevens Point  UnivTown
          Waukesha       UnivTown
          Whitewater     UnivTown
Wyoming   Laramie        UnivTown

[513 rows x 1 columns]

### Hypothesis: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (price_ratio=quarter_before_recession/recession_bottom)



In [65]:

def run_ttest():
    """First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is t ()rue or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss)."""



    univ_towns = (get_list_of_university_towns()
                  .assign(MarkUniv='UnivTown')
                  .set_index(["State", "RegionName"])
                 )
    univ_towns.sort_index(inplace=True)
    univ_towns.replace(value="Seattle", to_replace="University District", inplace=True)

    housing_data = convert_housing_data_to_quarters()
    housing_data.sort_index(inplace=True)



    univ_towns_housing = pd.merge(housing_data, univ_towns, how="left", left_index=True, right_index=True)
    univ_towns_housing.columns = univ_towns_housing.columns.astype('str')
    

    rec_start = get_recession_start().upper()
    rec_bottom = get_recession_bottom().upper()

    rec_start_previous = univ_towns_housing.columns.get_loc(rec_start) - 1
    qtr_before_rec_start =univ_towns_housing.columns[rec_start_previous]
    print("Quarter before recession start :", qtr_before_rec_start)
    
    univ_towns_housing['Ratio']=univ_towns_housing.apply(lambda x: x[qtr_before_rec_start]/x[rec_bottom],axis=1)

    
    non_univ_towns_housing = univ_towns_housing[univ_towns_housing['MarkUniv'].isnull()]
    univ_towns_housing = univ_towns_housing[univ_towns_housing['MarkUniv'] == 'UnivTown']

    tTestResult = ttest_ind(univ_towns_housing.loc[:,'Ratio'].dropna(), non_univ_towns_housing.loc[:,'Ratio'].dropna())

    testStatistic = tTestResult [0] 
    pValue = tTestResult [1] 
    print("T test result : "  , tTestResult)
    
    university_mean = univ_towns_housing['Ratio'].mean()
    nonuniversity_mean = non_univ_towns_housing['Ratio'].mean()
    
    better = 'university town' if university_mean < nonuniversity_mean else 'Non-university town'
    different = 'Yes' if pValue < 0.01 else 'No'

    return (different,pValue,better)

# #Ttest_indResult(statistic=-2.9195072956955008, pvalue=0.0035137908912504863)
run_ttest()

Quarter before recession start : 2008Q2
T test result :  Ttest_indResult(statistic=0.10935039821678649, pvalue=0.9129255377091339)


('No', 0.9129255377091339, 'Non-university town')